In [1]:
import numpy as np
import os
from PIL import Image# confirm mtcnn was installed correctly
import matplotlib.pyplot as plt
import mtcnn
import cv2

In [2]:
detector = mtcnn.MTCNN()

def detect_face(path_image,size=(160,160)):
    get_array = cv2.imread(path_image)
    get_array = cv2.cvtColor(get_array, cv2.COLOR_BGR2RGB)
    get_face = detector.detect_faces(get_array)
    if len(get_face) > 0:
        x1, y1, width, height = get_face[0]['box']
        x2, y2 = x1 + width, y1+ height
        face_array = get_array[y1:y2,x1:x2]
        image = Image.fromarray(face_array).resize(size)
        face_array_final = np.asarray(image)
        return face_array_final
    else: pass

In [3]:

img_path = 'data_image'
X = []
y = []
for celeb in os.listdir(img_path):
    path_ = img_path + '/' + celeb
    list_image = os.listdir(path_)
    if '.ipynb_checkpoints' in list_image:
        list_image.remove('.ipynb_checkpoints')
    else: pass    
    for img in list_image:
        full_path = path_ + '/' + img
        img_array = detect_face(full_path)
        if type(img_array) == np.ndarray:
            img_array_scaled = img_array/255
            y_label = celeb
            X.append(img_array_scaled)
            y.append(y_label)
        else: pass

In [4]:
X_array = np.array(X)
y_array = np.array(y)

# np.save('X_array.npy',X_array)
# np.save('y_array.npy',y_array)

# X_array = np.load('X_array.npy')
# y_array = np.load('y_array.npy')

In [5]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y_array)

y_ohe = pd.get_dummies(y_encoded).values

In [6]:
from scipy.spatial import distance
from keras.models import load_model
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

# load the model
model_facenet = load_model('facenet_model/facenet_keras.h5')

In [7]:
X_embed = model_facenet.predict(X_array)
X_embed_scaled = Normalizer(norm='l2').transform(X_embed)
# X_embed_scaled = StandardScaler().fit_transform(X_embed)

## Get mean distance

In [ ]:
mean_ikh = np.mean(X_embed_scaled[y_array == 'ikhbar'],axis=0)
mean_toni = np.mean(X_embed_scaled[y_array == 'toni'],axis=0)

np.save('mean_ikh.npy',mean_ikh)
np.save('mean_toni.npy',mean_toni)

## Classifier Model

In [8]:
#instantiate the model 
svc_clf = SVC(C=1,kernel='linear',probability=True)
#fit the model
svc_clf.fit(X_embed_scaled,y_array)

SVC(C=1, kernel='linear', probability=True)

In [19]:
logreg = LogisticRegression(C=1)
logreg.fit(X_embed_scaled,y_array)

LogisticRegression(C=1)

In [20]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_embed_scaled,y_array)

KNeighborsClassifier(n_neighbors=3)

In [21]:
gnb = GaussianNB()
gnb.fit(X_embed_scaled,y_array)

GaussianNB()

In [9]:
preds_svc_train = svc_clf.predict(X_embed_scaled)
# pred_logreg = logreg.predict(X_embed_scaled)
# pred_knn = knn.predict(X_embed_scaled)
# pred_gnb = gnb.predict(X_embed_scaled)

In [10]:
print('Train accuracy SVC:',accuracy_score(y_array,preds_svc_train))
# print('Train accuracy Logreg:',accuracy_score(y_array,pred_logreg))
# print('Train accuracy knn:',accuracy_score(y_array,pred_knn))
# print('Train accuracy gnb:',accuracy_score(y_array,pred_gnb))

Train accuracy SVC: 1.0


In [11]:
print(classification_report(y_array,preds_svc_train))
# print(20*'-')
# print(classification_report(y_array,pred_logreg))
# print(20*'-')
# print(classification_report(y_array,pred_knn))
# print(20*'-')
# print(classification_report(y_array,pred_gnb))

              precision    recall  f1-score   support

      ikhbar       1.00      1.00      1.00         8
        toni       1.00      1.00      1.00         6

    accuracy                           1.00        14
   macro avg       1.00      1.00      1.00        14
weighted avg       1.00      1.00      1.00        14



In [12]:
import pickle
with open('face_svc_ikh_toni.pkl', 'wb') as file_1:
  pickle.dump(svc_clf, file_1)

# with open('face_logreg_ikh_toni.pkl', 'wb') as file_2:
#   pickle.dump(logreg, file_2)

# with open('face_knn_ikh_toni.pkl', 'wb') as file_3:
#   pickle.dump(knn, file_3)

# with open('face_gnb_ikh_toni.pkl', 'wb') as file_4:
#   pickle.dump(gnb, file_4)